# Import and format phosphositeplus data

https://academic.oup.com/nar/article/47/D1/D433/5184725?login=true

https://www.phosphosite.org/ 

In [1]:
import pandas as pd
import numpy as np
import re

Due to the PhosphoSitePlus license, users need to [download](https://www.phosphosite.org/staticDownloads) the following files themselves prior to running this analysis.

In [2]:
df_phospho = pd.read_csv('../data/unformatted_ptm_data/Phosphorylation_site_dataset', sep='\t', skiprows=2)

In [3]:
df_ubi = pd.read_csv('../data/unformatted_ptm_data/Ubiquitination_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [4]:
df_sumo = pd.read_csv('../data/unformatted_ptm_data/Sumoylation_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [5]:
df_acetyl = pd.read_csv('../data/unformatted_ptm_data/Acetylation_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [6]:
df_methyl = pd.read_csv('../data/unformatted_ptm_data/Methylation_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [7]:
df_GalNAc = pd.read_csv('../data/unformatted_ptm_data/O-GalNAc_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [8]:
df_GlcNAc = pd.read_csv('../data/unformatted_ptm_data/O-GlcNAc_site_dataset', sep='\t', skiprows=2,low_memory=False)

In [9]:
df_regulatory = pd.read_csv('../data/unformatted_ptm_data/Regulatory_sites', sep='\t', skiprows=2,low_memory=False)

In [10]:
df_regulatory[0:3]

,GENE,PROTEIN,PROT_TYPE,ACC_ID,GENE_ID,HU_CHR_LOC,ORGANISM,MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,...,ON_FUNCTION,ON_PROCESS,ON_PROT_INTERACT,ON_OTHER_INTERACT,PMIDs,LT_LIT,MS_LIT,MS_CST,NOTES,Unnamed: 20
0,AQP2,AQP2,"Channel, misc.; Membrane protein, integral; Me...",P41181,359.0,12q13.12,human,K270-ub,999601,QSLPRGTkA______,...,protein degradation; intracellular localizatio...,NaN,NaN,NaN,21209006,6,0,0,stimulates phosphorylation of S261,NaN
1,Pdpk1,PDK1,"EC 2.7.11.1; KINASE; Kinase, protein; Nuclear ...",O55173,81745.0,10q12,rat,Y379-p,447695,DEDCYGNyDNLLSQF,...,"enzymatic activity, induced","translation, altered; cell adhesion, altered",NaN,NaN,18559349,8,1,0,Induces fibronectin expression and deposition ...,NaN
2,CD200R1,CD200R,"Membrane protein, integral; Receptor, misc.",Q8TD46,131450.0,3q13.2,human,Y302-p,458379,TEKNNPLyDTTNKVK,...,"molecular association, regulation",NaN,DOK2(INDUCES),NaN,19786546,2,2,11,recruitment of RasGAP by Dok2,NaN


In [11]:
def format_ptm_data(df_site):
    df = df_site.copy(deep=True)
    # subset to human
    df = df[df.ORGANISM=='human']
    # extract modified residue, modification type and position
    df['rsite'] = [re.sub('-.+','',i) for i in df['MOD_RSD']]
    df['mod'] = [re.sub('.*-','',i) for i in df['MOD_RSD']]
    df['mod_AA'] = [re.sub('\d+','',i) for i in df['rsite']]
    df['mod_position'] = [re.sub('[A-Z]+','',i) for i in df['rsite']]
    # subset to necessart information
    df = df[['ACC_ID','mod_AA','mod_position','mod']]
    df = df.rename(columns={"ACC_ID": "protein_id", "mod_AA": "AA", "mod_position": "position", "mod": "ptm"})
    # reset index
    df = df.reset_index(drop=True)
    
    return(df)

In [12]:
df_phospho_f = format_ptm_data(df_phospho)
df_ubi_f = format_ptm_data(df_ubi)
df_sumo_f = format_ptm_data(df_sumo)
df_acetyl_f = format_ptm_data(df_acetyl)
df_methyl_f = format_ptm_data(df_methyl)
df_GalNAc_f = format_ptm_data(df_GalNAc)
df_GlcNAc_f = format_ptm_data(df_GlcNAc)
df_regulatory_f = format_ptm_data(df_regulatory)

In [13]:
df_regulatory_f

,protein_id,AA,position,ptm
0,P41181,K,270,ub
1,Q8TD46,Y,302,p
2,P60484,S,380,p
3,P10636-8,T,231,p
4,Q9UPY3,S,1016,p
...,...,...,...,...
11229,P31749,T,308,p
11230,Q14247,K,198,ac
11231,Q9Y261,K,264,ac
11232,P16144,S,1494,p


## Count number of AA residues modified per PTM type to generate a ptm_site_dict

In [14]:
from collections import Counter

In [15]:
Counter(df_phospho_f.AA)

Counter({'T': 58664,
         'S': 141638,
         'Y': 39287,
         'H': 14,
         'R': 3,
         'V': 1,
         'N': 1,
         'P': 2,
         'F': 1,
         'G': 3,
         'L': 1,
         'K': 2,
         'A': 1,
         'I': 1,
         'D': 1})

In [16]:
Counter(df_ubi_f.AA)

Counter({'K': 97794, 'S': 1, 'R': 2, 'C': 2})

In [17]:
Counter(df_sumo_f.AA)

Counter({'K': 8442})

In [18]:
Counter(df_acetyl_f.AA)

Counter({'K': 22837, 'S': 10, 'T': 5})

In [19]:
Counter(df_methyl_f.AA)

Counter({'K': 5237, 'R': 11164, 'H': 3, 'C': 8, 'L': 1})

In [20]:
### There are multiple methylation types >> aggregate 

In [21]:
df_methyl_f.ptm.unique()

array(['m1', 'm2', 'me', 'm3'], dtype=object)

In [22]:
Counter(df_methyl_f[df_methyl_f.ptm=='m1'].AA)

Counter({'K': 4291, 'R': 9501, 'H': 3})

In [23]:
Counter(df_methyl_f[df_methyl_f.ptm=='m2'].AA)

Counter({'K': 533, 'R': 1566})

In [24]:
Counter(df_methyl_f[df_methyl_f.ptm=='me'].AA)

Counter({'R': 97, 'K': 80, 'C': 8, 'L': 1})

In [25]:
Counter(df_methyl_f[df_methyl_f.ptm=='m3'].AA)

Counter({'K': 333})

In [26]:
Counter(df_GalNAc_f.AA)

Counter({'S': 835, 'T': 1267, 'Y': 10})

In [27]:
Counter(df_GlcNAc_f.AA)

Counter({'S': 265, 'T': 187})

In [28]:
Counter(df_regulatory_f.AA)

Counter({'K': 1751,
         'Y': 1809,
         'S': 5567,
         'T': 1979,
         'R': 117,
         'L': 1,
         'I': 1,
         'N': 2,
         'H': 3,
         'C': 4})

In [29]:
Counter(df_regulatory_f.ptm)

Counter({'ub': 481,
         'p': 9330,
         'ac': 673,
         'm2': 90,
         'm1': 74,
         'sm': 490,
         'm3': 14,
         'me': 49,
         'gl': 26,
         'ng': 2,
         'pa': 4,
         'sc': 1})

### Rename regulatory PTM sites

In [30]:
# df_regulatory_f = df_regulatory_f[df_regulatory_f.ptm=='p'].reset_index(drop=True)
# df_regulatory_f['ptm'] = 'p_reg'

In [31]:
df_regulatory_f['ptm'] = df_regulatory_f['ptm']+'_reg'

In [32]:
df_regulatory_f

,protein_id,AA,position,ptm
0,P41181,K,270,ub_reg
1,Q8TD46,Y,302,p_reg
2,P60484,S,380,p_reg
3,P10636-8,T,231,p_reg
4,Q9UPY3,S,1016,p_reg
...,...,...,...,...
11229,P31749,T,308,p_reg
11230,Q14247,K,198,ac_reg
11231,Q9Y261,K,264,ac_reg
11232,P16144,S,1494,p_reg


In [33]:
ptm_site_dict = {'p':['S','T','Y'],
                 'ub':['K'],
                 'sm':['K'],
                 'ac':['K'],
                 'm':['K','R'],
                 'ga':['S','T'],
                 'gl':['S','T'],
                 'p_reg':['S','T','Y'],
                 'ub_reg':['K'],
                 'sm_reg':['K'],
                 'ac_reg':['K'],
                 'm_reg':['K','R'],
                 'ga_reg':['S','T'],
                 'gl_reg':['S','T'],}

## Concatenate ptm site tables and remove sites that are not according to the ptm_site_dict

In [34]:
ptm_df = None

for d in [df_phospho_f, df_ubi_f, df_sumo_f, df_acetyl_f, df_methyl_f, df_GalNAc_f, df_GlcNAc_f, df_regulatory_f]:
    if ptm_df is None:
        ptm_df = d
    else:
        ptm_df = pd.concat([ptm_df, d])

In [35]:
ptm_df

,protein_id,AA,position,ptm
0,P31946,T,2,p
1,P31946,S,6,p
2,P31946,Y,21,p
3,P31946,T,32,p
4,P31946,S,39,p
...,...,...,...,...
11229,P31749,T,308,p_reg
11230,Q14247,K,198,ac_reg
11231,Q9Y261,K,264,ac_reg
11232,P16144,S,1494,p_reg


### Aggregate different methylation types

In [36]:
ptm_df['ptm'] = ptm_df['ptm'].apply(lambda x: re.sub('me|m1|m2|m3','m',x) if x in ['m1','m2','me','m3','m1_reg','m2_reg','me_reg','m3_reg'] else x)

In [37]:
ptm_df['ptm'].unique()

array(['p', 'ub', 'sm', 'ac', 'm', 'ga', 'gl', 'ub_reg', 'p_reg',
       'ac_reg', 'm_reg', 'sm_reg', 'gl_reg', 'ng_reg', 'pa_reg',
       'sc_reg'], dtype=object)

In [38]:
### Remove regulatory ng, pa and sc

In [39]:
ptm_df = ptm_df[ptm_df.ptm.isin(['p', 'ub', 'sm', 'ac', 'm', 'ga', 'gl', 
                               'ub_reg', 'p_reg','ac_reg', 'm_reg', 'sm_reg', 'gl_reg'])]

In [40]:
ptm_df['ptm'].unique()

array(['p', 'ub', 'sm', 'ac', 'm', 'ga', 'gl', 'ub_reg', 'p_reg',
       'ac_reg', 'm_reg', 'sm_reg', 'gl_reg'], dtype=object)

In [41]:
### Subset ptm_df to AA sites in ptm_site_dict

In [42]:
def check_ptm_dict(mod, aa, ptm_d):
    pos_aa = ptm_d[mod]
    if aa in pos_aa:
        res = 1
    else: 
        res = 0
    return(res)

In [43]:
ptm_df['in_dict'] = ptm_df.apply(lambda x: check_ptm_dict(x['ptm'],x['AA'],ptm_site_dict), axis=1)

/var/folders/zx/n29r0swn0hddt1qgy159sjrw0000gn/T/ipykernel_15074/1158935549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptm_df['in_dict'] = ptm_df.apply(lambda x: check_ptm_dict(x['ptm'],x['AA'],ptm_site_dict), axis=1)


In [44]:
ptm_df = ptm_df[ptm_df.in_dict==1]

In [45]:
ptm_df = ptm_df.drop(columns=['in_dict']).reset_index(drop=True)

In [46]:
ptm_df

,protein_id,AA,position,ptm
0,P31946,T,2,p
1,P31946,S,6,p
2,P31946,Y,21,p
3,P31946,T,32,p
4,P31946,S,39,p
...,...,...,...,...
398827,P31749,T,308,p_reg
398828,Q14247,K,198,ac_reg
398829,Q9Y261,K,264,ac_reg
398830,P16144,S,1494,p_reg


In [47]:
#ptm_df[ptm_df.protein_id=='Q9UBS5']

## Format to wide table

In [48]:
ptm_oh = pd.get_dummies(ptm_df['ptm'])

In [49]:
ptm_annotation = ptm_df.join(ptm_oh)

In [50]:
ptm_annotation = ptm_annotation.drop(columns=['ptm'])

In [51]:
ptm_annotation

,protein_id,AA,position,ac,ac_reg,ga,gl,gl_reg,m,m_reg,p,p_reg,sm,sm_reg,ub,ub_reg
0,P31946,T,2,0,0,0,0,0,0,0,1,0,0,0,0,0
1,P31946,S,6,0,0,0,0,0,0,0,1,0,0,0,0,0
2,P31946,Y,21,0,0,0,0,0,0,0,1,0,0,0,0,0
3,P31946,T,32,0,0,0,0,0,0,0,1,0,0,0,0,0
4,P31946,S,39,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398827,P31749,T,308,0,0,0,0,0,0,0,0,1,0,0,0,0
398828,Q14247,K,198,0,1,0,0,0,0,0,0,0,0,0,0,0
398829,Q9Y261,K,264,0,1,0,0,0,0,0,0,0,0,0,0,0
398830,P16144,S,1494,0,0,0,0,0,0,0,0,1,0,0,0,0


## Aggregate rows 

In [52]:
ptm_annotation_grouped = ptm_annotation.groupby(['protein_id','AA','position'])

In [53]:
ptm_annotation_grouped = ptm_annotation_grouped.max()

In [54]:
ptm_annotation_grouped = ptm_annotation_grouped.reset_index()

In [55]:
ptm_annotation_grouped = ptm_annotation_grouped.reset_index(drop=True)

In [56]:
ptm_annotation_grouped

,protein_id,AA,position,ac,ac_reg,ga,gl,gl_reg,m,m_reg,p,p_reg,sm,sm_reg,ub,ub_reg
0,A0A024R5B6,K,43,0,0,0,0,0,0,0,0,0,0,0,1,1
1,A0A024RBG1,K,128,0,0,0,0,0,0,0,0,0,0,0,1,0
2,A0A024RBG1,K,134,0,0,0,0,0,0,0,0,0,0,0,1,0
3,A0A024RBG1,K,143,0,0,0,0,0,0,0,0,0,0,0,1,0
4,A0A024RBG1,K,5,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369206,Q9Y6Z7,K,97,1,0,0,0,0,0,0,0,0,0,0,0,0
369207,Q9Y6Z7,S,141,0,0,0,0,0,0,0,1,0,0,0,0,0
369208,Q9Y6Z7,T,155,0,0,0,0,0,0,0,1,0,0,0,0,0
369209,Q9YNA8,S,465,0,0,0,0,0,0,0,1,0,0,0,0,0


In [57]:
#ptm_annotation_grouped[ptm_annotation_grouped.protein_id=='Q9UBS5']

## Add phosphosite to p if there is an annotated regulatory phosphosite

Apparently there are some misannotations of there regulatory sites in phosphositeplus

In [58]:
print(ptm_annotation_grouped[ptm_annotation_grouped.p==1].shape[0])
print(ptm_annotation_grouped[ptm_annotation_grouped.p_reg==1].shape[0])
print(ptm_annotation_grouped[(ptm_annotation_grouped.p==0) & (ptm_annotation_grouped.p_reg==1)].shape[0])

239588
9324
5


In [59]:
ptm_annotation_grouped['p'] = np.where(ptm_annotation_grouped.p_reg == 1, 1, ptm_annotation_grouped.p)
ptm_annotation_grouped['ub'] = np.where(ptm_annotation_grouped.ub_reg == 1, 1, ptm_annotation_grouped.ub)
ptm_annotation_grouped['ac'] = np.where(ptm_annotation_grouped.ac_reg == 1, 1, ptm_annotation_grouped.ac)
ptm_annotation_grouped['sm'] = np.where(ptm_annotation_grouped.sm_reg == 1, 1, ptm_annotation_grouped.sm)
ptm_annotation_grouped['gl'] = np.where(ptm_annotation_grouped.gl_reg == 1, 1, ptm_annotation_grouped.gl)

In [60]:
print(ptm_annotation_grouped[ptm_annotation_grouped.p==1].shape[0])
print(ptm_annotation_grouped[ptm_annotation_grouped.p_reg==1].shape[0])
print(ptm_annotation_grouped[(ptm_annotation_grouped.p==0) & (ptm_annotation_grouped.p_reg==1)].shape[0])

239593
9324
0


In [61]:
# ptm_annotation_grouped[ptm_annotation_grouped.protein_id=='Q9UBS5']

In [62]:
ptm_annotation_grouped[0:3]

,protein_id,AA,position,ac,ac_reg,ga,gl,gl_reg,m,m_reg,p,p_reg,sm,sm_reg,ub,ub_reg
0,A0A024R5B6,K,43,0,0,0,0,0,0,0,0,0,0,0,1,1
1,A0A024RBG1,K,128,0,0,0,0,0,0,0,0,0,0,0,1,0
2,A0A024RBG1,K,134,0,0,0,0,0,0,0,0,0,0,0,1,0


## Write outfile

In [63]:
ptm_annotation_grouped.to_csv('../data/ptm_data/phosphositeplus_annotation.csv', index=False)

In [64]:
ptm_annotation_grouped[ptm_annotation_grouped.p_reg==1].shape[0]

9324

In [65]:
ptm_annotation_grouped[ptm_annotation_grouped.ub_reg==1].shape[0]

481

In [66]:
ptm_annotation_grouped[ptm_annotation_grouped.ac_reg==1].shape[0]

668

In [67]:
ptm_annotation_grouped[ptm_annotation_grouped.sm_reg==1].shape[0]

490

In [68]:
ptm_annotation_grouped[ptm_annotation_grouped.gl_reg==1].shape[0]

26